In [1]:
#quando gastas ao mes
monthlyspending=1000

#money earned monthly not from salary, money that doest need work it comes from other sources
#atualmente contem, bondora, dinheiro do unity youtube e mintos
monthlypassive=180

#monthly invested - is equivalent to amount of money earned monthly through salary that you save, or better, invest
PMT=1000 


#all my money
currentMoney=51044.91

#revolut invest, degiro,  e trading
currentInvested =20400

#imposto sobre o que la esta ao ano - interest on invested money
annualrate=0.08

#METRICAS IMPORTANTES: QUANDO È QUE ESTES SE CRUZAM?
#moneyneededtolivewithpassive
#networthinvestedwopassive

In [2]:
import datetime
import altair as alt



today = datetime.datetime.today()

#qunado vais morrer
dayidie = datetime.datetime(2086,1,1)

lifetime = dayidie-today
monthstolive = int(lifetime.days/30.2)




currentMoney=currentMoney-currentInvested



data={
    'moneyneededtolive':[],
    'networth':[],
    'networthwpassive':[],
    'networthwinvested':[],
    'currentinvested':[],
    'moneyneededtolivewithpassive':[],
    'networthinvestedwopassive':[],
    'x':[]
}

yearstocalculate = 30

seriesofmonths = range(1,monthstolive)

for i in seriesofmonths:
    
    
    if(i/12>yearstocalculate):
        break
    
    #ano
    data['x'].append(24+i/12)#i)
    
    #quantia pa viver
    data['moneyneededtolive'].append((monthlyspending)*(monthstolive-i))
    #quantia pa viver se ganhares passivo
    data['moneyneededtolivewithpassive'].append((monthlyspending-monthlypassive)*(monthstolive-i))
    
    #isto é o juro composto  sobre o dinheiro que investi ate agora.
    #valor estimado do dinheiro investiod (quanto cresce ao mes/ano)
    cumulativeInvested = currentInvested*(1+i*(annualrate/12))#atual mais ganhos mensais incrementais
    data['currentinvested'].append(cumulativeInvested)
    
    #isto é o juro composto  sobre o dinheiro que investirei.
    #assumindo que invisto PMT, todos os meses, em que é que isso se traduz ao longo do tempo
    cumulativePMTearnings = PMT*i*(1+(annualrate/12))
    
    
    #isto é o dinehiro que recebo não é investido
    #simboliza o dinheiro que seria gasto diretamente no mes em que era recebudo
    cumulativePassive =monthlypassive*i
    
    #net worth estimado, proveniente de invers
    data['networth'].append(currentMoney+cumulativePMTearnings)#+cumulativePassive)
    
    #quanto dinheiro vais ter ao longo de tempo 
    #nao conta com dinehri passivo, pois e como se este fosse automaticamente abatido no mes em que é recebido
    #assume que o monthlypassive < monthly spend.
    #se alguma vez esta condicao for quebrada, quer dizer que é self suficient e nao precisas de trabalhar
    data['networthinvestedwopassive'].append(currentMoney+cumulativePMTearnings+cumulativeInvested)
    
    data['networthwpassive'].append(currentMoney+cumulativePMTearnings+cumulativePassive)
    data['networthwinvested'].append(currentMoney+cumulativePMTearnings+cumulativePassive+cumulativeInvested)
    
    
    
import pandas as pd
    
#d = {'x': seriesofyears, 'Needed Money': moneyneededtolive,'Net Worth':networth}
#df = pd.DataFrame(data=d)
df=pd.DataFrame.from_dict(data)

dfmelted= df.melt('x', var_name='Series', value_name='y')

In [3]:


source=dfmelted

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')



# The basic line
line = alt.Chart(source).mark_line().encode(
    x='x:Q',
    y='y:Q',
    color='Series:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='x:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'y:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='x:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)


alt.LayerChart(...)

In [4]:
dfmelted.to_csv("modelv2.csv")

In [5]:

first = df.iloc[[0]]['x'].values[0]
whatage=df[df['networthinvestedwopassive']>=df['moneyneededtolivewithpassive']].iloc[[0]]['x'].values[0]

inHowManyYears=whatage-first

inHowManyYears

25.166666666666668